<p style="font-size: 20px; color: white; margin: 0; ">NS4300/6300 Homework 5</p>
<p style="font-size: 14px; color: white; margin: 0;">Due March 25th by 11:59PM</p>
<p style="font-size: 14px; color: white; margin: 0;">Homework should be completed by modifying this .ipynb file and submitting it on Gradescope.</p>
<p style="font-size: 14px; color: white; margin: 0;">The submitted .ipynb file should have all cells executed and answers to questions where indicated.</p>

<p style="font-size: 20px; color: orange; margin: 0 ">1.) Create a pandas data frame with thesubcutaneous adipose tissue and skeletal muscle tissue TPM transcriptomic data from GTEx. Normalize the rows to the row mean, normalize the columns to the column median, and then log2 transform. Print the first 5 rows and first 5 columns as output in this notebook file.</p>

In [9]:
import pandas as pd
import numpy as np
from pdb import set_trace as st
from copy import copy
import seaborn as sns
import fastcluster
from scipy.cluster.hierarchy import linkage, dendrogram
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from scipy import stats

from matplotlib.patches import Patch
from sklearn.manifold import TSNE #need scikit-learn installed
from sklearn.decomposition import PCA

from statsmodels.stats.multitest import multipletests

In [10]:
# Data Available at: https://www.gtexportal.org/home/downloads/adult-gtex/bulk_tissue_expression
# Sample Annotations Available at: https://gtexportal.org/home/downloads/adult-gtex/metadata

# Print the first 5 lines of the RNA seq file
RNA_File = 'GTEx_Analysis_v10_RNASeQCv2.4.2_gene_tpm.gct'
# Print the first 5 lines of the RNA seq file
with open(RNA_File, 'r') as read_file, open('small_file.txt', 'w') as write_file:  
    # i keeps track of which line we are on
    i=0
    # iterating through the file will iterate line by line automatically
    #   that is, the iteration variable will be individual lines of the file
    for line in read_file:
        print(line)
        i=i+1
        if i>=5:
            break

# Data available at:
# GTEx main page -> Downloads in top dropdown menu -> Open access data -> Bulk tissue expression tab -> Select the Gene TPMs file
# Meta data available under the Meta data tab. Select the file with the description "A de-identified, open access version of the sample annotations available in dbGaP."

#1.2

59033	19616

Name	Description	GTEX-1117F-0005-SM-HL9SH	GTEX-1117F-0011-R10b-SM-GI4VE	GTEX-1117F-0011-R11b-SM-GIN8R	GTEX-1117F-0011-R2b-SM-GI4VL	GTEX-1117F-0011-R3a-SM-GJ3PJ	GTEX-1117F-0011-R4b-SM-GI4VM	GTEX-1117F-0011-R5a-SM-GI4VW	GTEX-1117F-0011-R6a-SM-GI4VX	GTEX-1117F-0011-R7a-SM-H65ZK	GTEX-1117F-0226-SM-5GZZ7	GTEX-1117F-0426-SM-5EGHI	GTEX-1117F-0526-SM-5EGHJ	GTEX-1117F-0626-SM-5N9CS	GTEX-1117F-0726-SM-5GIEN	GTEX-1117F-1426-SM-H65ZH	GTEX-1117F-2126-SM-GIN9G	GTEX-1117F-2426-SM-5EGGH	GTEX-1117F-2526-SM-5GZY6	GTEX-1117F-2826-SM-5GZXL	GTEX-1117F-2926-SM-5GZYI	GTEX-1117F-3026-SM-5GZYU	GTEX-1117F-3226-SM-5N9CT	GTEX-111CU-0002-SM-HTQ8G	GTEX-111CU-0005-SM-GJ3PH	GTEX-111CU-0126-SM-5GZWZ	GTEX-111CU-0226-SM-5GZXC	GTEX-111CU-0326-SM-5GZXO	GTEX-111CU-0426-SM-5GZY1	GTEX-111CU-0526-SM-5EGHK	GTEX-111CU-0626-SM-5EGHL	GTEX-111CU-0726-SM-5GZYD	GTEX-111CU-0826-SM-5EGIJ	GTEX-111CU-0926-SM-5EGIK	GTEX-111CU-1026-SM-5EGIL	GTEX-111CU-1126-SM-5EGIM	GTEX-111CU-1226-SM-5EGIN	GTEX-111CU-1326-SM-5NQ8L	GTEX-

In [11]:
# Import the first five rows of the RNA_Seq file into a data frame
#   Skip the 0th and 1st row because they do not contain data as seen in the printed first several lines
mRNA_top = pd.read_csv(RNA_File, delimiter='\t', skiprows=[0,1], nrows=5)
mRNA_top.iloc[range(5), range(5)]

,Name,Description,GTEX-1117F-0005-SM-HL9SH,GTEX-1117F-0011-R10b-SM-GI4VE,GTEX-1117F-0011-R11b-SM-GIN8R
0,ENSG00000223972.5,DDX11L1,0.00000,0.000000,0.000000
1,ENSG00000227232.5,WASH7P,1.33343,3.579280,10.189300
2,ENSG00000278267.1,MIR6859-1,0.00000,0.000000,0.000000
3,ENSG00000243485.5,MIR1302-2HG,0.00000,0.093825,0.034191
4,ENSG00000237613.2,FAM138A,0.00000,0.000000,0.000000


In [12]:
# Open the sample annotations
#  The annotations file is much smaller so we can open it directly into a dataframe and examine the header and first few rows
Annotations_File = 'GTEx_Analysis_v10_Annotations_SampleAttributesDS.txt'
Annotations = pd.read_csv(Annotations_File, delimiter='\t')
Annotations.iloc[range(0,5),range(0,5)]

/var/folders/7c/v6941zjx7r54gww6_y54czmw0000gn/T/ipykernel_6203/3806524899.py:4: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  Annotations = pd.read_csv(Annotations_File, delimiter='\t')


,SAMPID,SMATSSCR,SMCENTER,SMPTHNTS,SMRIN
0,BMS-X4LF-0126-SM-4JBHL,NaN,B1,NaN,7.5
1,BMS-X4LF-0226-SM-4JBJ3,NaN,B1,NaN,6.9
2,BMS-X4LF-0326-SM-4JBIR,NaN,B1,NaN,7.4
3,BMS-X4LF-0426-SM-4JBIS,NaN,B1,NaN,7.1
4,BMS-X4LF-0526-SM-4JBHX,NaN,B1,NaN,8.8


In [14]:
# The tissue designations are stored in the 'SMTSD' column
#   We can use the unique() function to see all the unique tissues
Unique_Tissues = Annotations['SMTSD'].unique()
Unique_Tissues = sorted(Unique_Tissues)
for item in Unique_Tissues:
    print(item)

Adipose - Subcutaneous
Adipose - Visceral (Omentum)
Adrenal Gland
Artery - Aorta
Artery - Coronary
Artery - Pulmonary
Artery - Tibial
Bladder
Brain - Amygdala
Brain - Anterior cingulate cortex (BA24)
Brain - Caudate (basal ganglia)
Brain - Cerebellar Hemisphere
Brain - Cerebellum
Brain - Cortex
Brain - Frontal Cortex (BA9)
Brain - Hippocampus
Brain - Hypothalamus
Brain - Nucleus accumbens (basal ganglia)
Brain - Putamen (basal ganglia)
Brain - Spinal cord (cervical c-1)
Brain - Substantia nigra
Breast - Mammary Tissue
Cells - Cultured fibroblasts
Cells - EBV-transformed lymphocytes
Cells - Leukemia cell line (CML)
Cervix - Ectocervix
Cervix - Endocervix
Colon - Sigmoid
Colon - Transverse
Colon - Transverse - Mixed Cell
Colon - Transverse - Mucosa
Colon - Transverse - Muscularis
Esophagus - Gastroesophageal Junction
Esophagus - Mucosa
Esophagus - Muscularis
Fallopian Tube
Heart - Atrial Appendage
Heart - Left Ventricle
Kidney - Cortex
Kidney - Medulla
Liver
Liver - Hepatocyte
Liver - Mi

In [15]:
# Our task now is to extract the samples we want from the large RNA seq file
#   We can specify a tissue and extract samples only from that tissue
#Tissue_of_Interest = ['Liver', 'Kidney - Cortex']
Tissue_of_Interest = ['Liver', 'Muscle - Skeletal']
Sample_Names = Annotations['SAMPID']
Tissue_Types = Annotations['SMTSD']
n_samples = len(Sample_Names)
Annotations_DF = pd.DataFrame({'tissue': Tissue_Types.to_numpy(), 'sample': Sample_Names.to_numpy()}, index=Sample_Names)

# We can use a list comprehension to create a list of indices of the samples that are of the tissue type we are interested in
#   Liver_Sample_Indices will be a list of True/False values
ToI_Sample_Indices = [Tissue_Types[i] in Tissue_of_Interest for i in range(n_samples)]

# We can use the list of indices to extract the sample names that are of the tissue type we are interested in
ToI_Samples = Sample_Names[ToI_Sample_Indices]
ToI_Samples = np.array(ToI_Samples)

# Find the column locations of the tissue of interest
RNA_Seq_DataSet_Columns = mRNA_top.columns

# Find the column locations of the tissue of interest
#   Use list comprehension to create a list of True/False values representing whether each column in the RNA Seq data set is a liver sample
#   In this example, x is iterating through the columns of the RNA_Seq_DataSet_Columns
Colums_to_Keep_Indices = [x in np.array(ToI_Samples) for x in RNA_Seq_DataSet_Columns]

# Print how many samples there are of the tissue of interest
#   The sum of the list of True/False values will give the number of True values (True=1, False=0)
n_tissue = sum(Colums_to_Keep_Indices)
print('Number of samples: ' + str(n_tissue))

# Keep the gene symbol column as well
#   The column with index 1 (2nd column) contains the gene symbol. We want to keep this column as well and it would not have been included in the previous list of True/False values (it's not a liver sample)
Colums_to_Keep_Indices[1] = True
Columns_to_Keep = RNA_Seq_DataSet_Columns[Colums_to_Keep_Indices]

Tissue_mRNA = pd.read_csv(RNA_File, skiprows=[0,1], usecols=Columns_to_Keep, delimiter='\t')

# Specify the row names as the gene symbols
Tissue_mRNA.index = Tissue_mRNA.loc[:,'Description']

# Remove the column that repeats the row names
Tissue_mRNA.drop('Description', inplace=True, axis=1)

# Remove the name associated with the row names
#   This has no functional purpose but it makes the data frame look cleaner - it's a matter of personal preference
Tissue_mRNA.rename_axis(None, inplace=True, axis=0)

# Remove duplicate indices
Tissue_mRNA = Tissue_mRNA[~Tissue_mRNA.index.duplicated(keep='first')]

# Normalize and transform the transcriptomics data frame
Tissue_mRNA = Tissue_mRNA.dropna()
Tissue_mRNA = Tissue_mRNA.replace(0, 0.125)
Tissue_mRNA_RowNorm = Tissue_mRNA.div(Tissue_mRNA.mean(axis=1), axis=0)
Tissue_mRNA_ColNorm = Tissue_mRNA_RowNorm.div(Tissue_mRNA_RowNorm.median(axis=0), axis=1)
Tissue_mRNA_Log2 = np.log2(Tissue_mRNA_ColNorm)

Tissue_mRNA_Log2.iloc[range(5), range(5)]

Number of samples: 1080


,GTEX-1117F-0426-SM-5EGHI,GTEX-111CU-2026-SM-5GZZC,GTEX-111FC-0326-SM-5GZZ1,GTEX-111FC-1226-SM-HAV27,GTEX-111VG-2626-SM-5GZY2
DDX11L1,0.417026,0.417285,-2.322039,0.429635,0.425556
WASH7P,1.141021,-0.735991,-0.526502,0.921451,-2.041965
MIR6859-1,-0.039106,-0.038847,-0.007499,-0.026498,-0.030576
MIR1302-2HG,0.301537,0.301796,0.333144,-2.093555,0.310067
FAM138A,0.270130,0.270389,-1.964662,0.282738,-1.895154


<p style="font-size: 20px; color: orange; margin: 0 ">2.) Create a heat map where only the columns (corresponding to the tissue samples) are clustered. Be sure to specify the tissue type of the samples with a column color bar. If your computer takes more than 4 minutes to perform the clustering, you can decrease the number of samples in the dataset by selecting 50 samples of each tissue type.</p>

<p style="font-size: 20px; color: orange; margin: 0 ">3.) Perform dimensional reduction (to 2 dimensions) on the samples using TSNE. Make a scatter plot of the TSNE values. Color the dots representing the samples by the tissue type.</p>

<p style="font-size: 20px; color: orange; margin: 0 ">4.) Perform dimensional reduction (to 2 dimensions) on the samples using principal component analysis. Make a scatter plot of the principal components. Color the dots representing the samples by the tissue type.</p>

<p style="font-size: 20px; color: orange; margin: 0 ">5.) Create a volcano plot where the log2-transformed ratio of transcript abundance in subcutaneous adipose tissue over skeletal muscle tissue is represented on the x-axis and the -log10 of the benjamini-hochberg corrected p-value is on the y-axis. If a adjusted p-value is less than or equal to 1e-300, set it equal to 1e-300.</p>

<p style="font-size: 20px; color: orange; margin: 0 ">6.) Perform a GSEA Preranked Enrichment analysis on the transcripts. How could the transcripts be ordered when passed to the Preranked function? Use h.all as the gene set. How many genes classified as part of cholesterol homeostasis are part of the core enrichment when looking at upregulation in adipose tissue?</p>

<p style="font-size: 20px; color: orange; margin: 0 ">7.) Use Enrichr to perform an enrichment analysis on KEGG pathways of the top 1000 transcripts up-regulated in subcutaneous adipose tissue. Are the findings consistent with known biology?</p>

<p style="font-size: 20px; color: orange; margin: 0 ">NS 6300 Question) Why are there so many significantly regulated transcripts and why are p-values so small?</p>